# HPDM097 - Simulation Research Project

---

### Abstract

This project replicates a Discrete Event Simulation (DES) model from Monks et al. [[1]](https://doi.org/10.1186/s12913-016-1789-4) for capacity planning in acute and community stroke services. The original model, built in SIMUL8, analysed stroke admissions and system bottlenecks across patient pathways. Our Python based recreation focused on arrivals to an Acute Stroke Unit (ASU), length of stay (LOS) and discharge routing, using exponential and lognormal distributions for arrivals and bed occupancy.

Through seven development iterations, we incorporated warm-up analysis, multiple replications and scenario testing. The model reproduced the original study’s key trend: increasing bed capacity non-linearly reduces patient delays. 

Our empirical delay estimates were higher than those published due to methodological differences: our model measured delays via simulation, recording any patient who waited, while the published results used analytical approximations such as the Erlang loss formula.

Key challenges included interpreting logic from a proprietary simulation tool and simplifying the model scope. We opted not to use a Large Language Model (LLM), instead prioritising hands-on skill development. The project demonstrated that transparent documentation significantly supports reproducibility.

---

### Introduction

This project aims to replicate the DES model described in the paper *“A Modelling Tool for Capacity Planning in Acute and Community Stroke Services.”* The original model explored capacity needs across acute and community services for over 630 stroke admissions annually. The authors investigated current bottlenecks affecting patient flow, projected future capacity needs under increased admissions, the potential benefits of co-locating and pooling the acute and rehabilitation units and the impact of various patient subgroups on overall capacity demands.

In our replication, we adopted the core logic of patients arriving at an ASU and then transferring onward to different destinations. Due to scope and practical considerations, our design includes several simplifications.

1. **Arrivals:** We model multiple types of stroke admissions, each arriving according to an exponential distribution with a unique mean.
2. **Length of Stay (LOS):** Once admitted to the ASU, patients occupy a bed for a duration sampled from a lognormal distribution, consistent with the paper’s approach. 
3. **Destinations:** Upon completion of their LOS in the ASU, patients exit to “Other”, transfer to Early Supported Discharge (ESD), or move to Rehabilitation. We did not model post-ASU services beyond their immediate impact on an ASU beds.

By focusing on this core patient flow, our work remains faithful to the essential mechanics of the original paper can be seen in Fig 1. while remaining feasible within our scope, the available model documentation and time we had. The diagram of what we have replicated can be seen in Fig 2.

<img src="images/asu_model.png" alt="Acute Stroke Unit Original Model" width="500" height="300">
*Figure 1. Acute Stroke Unit model diagram*
<img src="images/replicated_model.png" alt="Acute Stroke Unit Replicated Model" width="500" height="300">
*Figure 2. Our replication of the model*

---

### Methods

We followed an iterative development approach and as mentioned previously, deliberately avoided LLM use to deepen our understanding of DES principles. The project evolved over seven development iterations, with pair programming used extensively to ensure continuity.

**Iteration 1:**  
Built a basic patient arrival generator for a single, generic patient type and an ASU class using hardcoded exponential arrivals and lognormal LOS. The output seen in Fig 3. 

<img src="images/dataframe_op.png" alt="Iteration 1 Output" width="350" height="125">
*Figure 3. Iteration 1 output of model.*


**Iteration 2:**  
Introduced four distinct patient types, each with its own arrival rate. We decided to have a single stroke patient type, deviating from the papers three patient types. A single-run function and Experiment class were added to manage parameters and execution.
| **Patient Type**         |
|--------------------------|
| **Stroke**               |
| **TIA**                  |
| **Complex-neurological** |
| **Other**                |

*Table 1. Patient Types*

**Iteration 3:**  
Introduced tracking of patient types using dictionaries, allowing us to monitor the flow of each patient type and validate proportions by switching patient types on and off.

**Iteration 4:**  
Unified arrival logic, added parameterised distributions and seeded random number streams for reproducibility.

**Iteration 5:**  
Implemented discharge routing post LOS using defined probabilities. Patients were directed to one of three routes summarised in the table below:

| **Destination**                  | **Stroke (%)** | **TIA (%)** | **Complex Neurological (%)** | **Other (%)** |
|----------------------------------|---------------:|------------:|-----------------------------:|--------------:|
| **Rehabilitation**               | 24             | 1           | 11                           | 5             |
| **Early Supported Discharge (ESD)** | 13          | 1           | 5                            | 10            |
| **Other**                        | 63             | 98          | 84                           | 85            |

*Table 2. Breakdown of transfer %*

**Iteration 6:**  
An Audit class was implemented to record metrics such as LOS, delay probability and bed occupancy. We developed a multiple replication function and created scenario-specific result summaries for easier analysis and added visualisations.

**Iteration 7:**  
Added a warm-up period to eliminate initialisation bias and added three *'What if'* scenario experiments - these can be found in Table 5 and 6. This final iteration also included polishing outputs, commenting the code, developing visualisations and building a simple Streamlit app. Our GitHub repository was reorganised, to separate core code from archived iterations.

Pair programming was valuable throughout where frequent logic updates required continuous collaboration. By the end, our model was fully functional, linted and traceable. Sources used in the build of this project included the Github repository of Health Data Science and Operations Research [[2]](https://github.com/health-data-science-OR/stochastic_systems) and Streamlit's online documentation [[3]](https://docs.streamlit.io/).

Metrics such as delay probability and utilisation were collected only after the system reached steady-state, verified through time-series inspection. We also ran multiple replications and applied a confidence interval-based method to determine the minimum required replications for 5% precision, results suggested this was at six replications. These practices align with standard DES methodology recommended by Law [[4]](https://industri.fatek.unpatti.ac.id/wp-content/uploads/2019/03/108-Simulation-Modeling-and-Analysis-Averill-M.-Law-Edisi-5-2014.pdf).

---

### Results

The final simulation model effectively replicates the core structure and logic of the original study. It simulates an Acute Stroke Unit in which four types of patient's stroke, Transient Ischemic Attack (TIA), complex neurological issues and other conditions arrive randomly. These patients occupy a shared bed resource for a given length of stay that follows a lognormal distribution. They are then routed to different discharge destinations based on defined probabilities. 

The model tracks performance metrics such as delay probability and bed occupancy. It supports single runs, multiple replications, warm-up analysis and scenario-based experimentation. The development process involved multiple iterations to ensure logical consistency and adherence to the methodology outlined in the original study.

#### Model Iterations Testing

| Iteration    | Description                                                                 | Test Details                                                |
|--------------|-----------------------------------------------------------------------------|-------------------------------------------------------------|
| Iteration 1  | Basic single-patient model                                    | Output confirmed correct event sequencing                  |
| Iteration 2  | Arrival processes for all types                               | Validated correct event order using printed simulation logs                  |
| Iteration 3  | Patient type tracking and subgroup monitoring                 | Test percentiles of total patient type confirmed           |
| Iteration 4  | Unified arrival logic, parameterised types, seeded streams    | Test percentiles and output confirmed validity             |
| Iteration 5  | Routing logic added post-ASU stay                             | Patient journey validation test confirmed                  |
| Iteration 6  | Audit class, multiple replications, result plotting added     | Audit metrics and replication outputs confirmed            |
| Iteration 7  | Warm-up period and scenario analysis implemented              | Scenario outputs and warm-up reliability confirmed         |

*Table 3. Iteration test details*


In the Simulation Model, we developed a single_run(), multiple_replications() and a multiple_run() function each with their own unique use cases below we will demonstrate the use of these functions and their outputs.

#### Example Simulation Output (10 Beds, 1 Replication) using `single_run()` function
**`single_run()`**
- Delay probability: 0.61
- Bed Utilisation: 0.87

#### Example Simulation Output (10 Beds, 5 Replications) using `multiple_replications()` function
**`multiple_replications()`**
- Runs the **same simulation setup** multiple times (replications)
- Helps reduce variability due to randomness in simulation
- Typically used to evaluate **single bed capacity** 
- Returns averaged metrics such as:
  - Delay probability
  - Bed utilisation
  - Average number of patients

#### Example Simulation Output of Bed Range (10 to 17)  

*Each with 150 Replications using `multiple_run()` function*
- Warm up period: 3 years
- Collection period: 5 years
- Scenario (base case)

| Bed Capacity | Avg. Patients| Overall Bed Utilization| Overall Delay Probability| 1 in Every n Patients|
|--------------|-------------:|------------------------:|--------------------------:|-------------------:|
| 9            | 2786         | 0.96                    | 0.86                      | 1.0                |
| 10           | 2788         | 0.86                    | 0.57                      | 2.0                |
| 11           | 2789         | 0.79                    | 0.36                      | 3.0                |
| 12           | 2789         | 0.72                    | 0.22                      | 5.0                |
| 13           | 2789         | 0.66                    | 0.13                      | 8.0                |
| 14           | 2789         | 0.62                    | 0.07                      | 14.0               |
| 15           | 2789         | 0.58                    | 0.04                      | 27.0               |
| 16           | 2789         | 0.54                    | 0.02                      | 55.0               |
| 17           | 2789         | 0.51                    | 0.01                      | 114.0              |

*Table 4. Average number of patients, bed utilisation and delay probability for different bed capacities*

**`multiple_run()`**
- Runs the **same simulation setup** multiple times (replications)
- Helps reduce variability due to randomness in simulation
- Typically used to evaluate a **Range of bed capacities** 
- Returns averaged metrics such as:
  - Delay probability
  - Bed occupancy
  - Number of patients processed


### **Interpretation of model results**


**Simulation Results – Delay Probability (10–14 Beds)**

| **Beds** | **Current Admissions** | **No Complex Neuro**  | **+5% Admissions** |
|----------|------------------------|-----------------------|--------------------|
| 10       | 0.57 (1 in 2)          | 0.32 (1 in 3)         | 0.47 (1 in 2)      |
| 11       | 0.36 (1 in 3)          | 0.19 (1 in 5)         | 0.28 (1 in 4)      |
| 12       | 0.22 (1 in 5)          | 0.10 (1 in 10)        | 0.17 (1 in 6)      |
| 13       | 0.13 (1 in 8)          | 0.05 (1 in 19)        | 0.09 (1 in 11)     |
| 14       | 0.07 (1 in 14)         | 0.03 (1 in 38)        | 0.05 (1 in 20)     |

*Table 5. Replicated DES Results*


**Published Results: Delay Probability (10–14 Beds)**

| **Beds** | **Current Admissions** | **No Complex Neuro**  | **+5% Admissions** |
|----------|------------------------|-----------------------|--------------------|
| 10       | 0.14 (1 in 7)          | 0.09 (1 in 11)        | 0.16 (1 in 6)      |
| 11       | 0.09 (1 in 11)         | 0.05 (1 in 18)        | 0.11 (1 in 9)      |
| 12       | 0.06 (1 in 16)         | 0.03 (1 in 32)        | 0.07 (1 in 13)     |
| 13       | 0.04 (1 in 28)         | 0.02 (1 in 55)        | 0.05 (1 in 21)     |
| 14       | 0.02 (1 in 50)         | 0.01 (1 in 100)       | 0.03 (1 in 34)     |

*Table 6. Published DES Results*


The simulation reproduced the trend seen in the published results: delay probability decreases as ASU bed capacity increases. Scenarios with 5% higher admissions resulted in higher delays, while removing complex neuro patients led to lower delays. Overall the delay probabilities were higher in the simulation, this difference is expected due to methodology — the published results used the Erlang loss formula, whereas the simulation estimates delays empirically (i.e., any patient with wait time > 0).

Figure 4. Delay Probability Comparison

<img src="results/comparison_delay_probability.png" alt="Delay Probability Comparison" width="900" height="400">

---

### Discussion

This hands-on recreation of Monks et al. [[1]](https://doi.org/10.1186/s12913-016-1789-4) DES model proved to be both educational and rewarding. The original study was a standout example in healthcare simulation reporting, providing a detailed explanation of the model’s context, purpose and assumptions. The supplementary appendix was invaluable in helping us design a simplified yet representative recreation in Python.

Despite the paper’s strengths, replication posed some challenges. The original model was built using SIMUL8, a proprietary visual simulation tool. As Python users, we had to interpret and reimplement aspects of the model that may have been handled implicitly by the software interface such as conditional logic for ward transfers or pooled bed allocations. This required informed assumptions and highlighted the limits of reproducing the model exactly without direct access to the original implementation.

Monks et al. [[1]](https://doi.org/10.1186/s12913-016-1789-4) went beyond most DES studies in terms of documentation. The inclusion of clear scenario definitions, parameter tables and conceptual logic greatly supported our efforts. A few additions such as pseudocode, flow diagrams, or open-source code would further enhance reusability. Nonetheless, the paper stands out for its transparency and we found it immensely valuable both as a modelling case study and as a teaching tool.

It’s worth noting that some challenges we faced stemmed from our limited experience with discrete-event simulation. Concepts like warm-up periods, replication design and simulation variability were new to us. To maintain scope, we excluded modelling of the rehabilitation unit beyond its influence on ASU capacity.

Due to our model measuring delays empirically (any patient waiting was counted), it produced slightly higher delay estimates than the analytical Erlang-based approach in the original study. Despite that, our model successfully captured the same directional trends such as the non-linear relationship between bed capacity and delays and the impact of patient mix.

Comparing to other healthcare DES studies, such as those by Yakutcan et al. [[5]](https://doi.org/10.1111/1475-6773.13652) and Demir et al. [[6]](https://doi.org/10.1057/hs.2016.1), we found that while methodologically strong, they lacked the documentation needed for easy replication. Both used proprietary tools and omitted code-level details. In contrast, Monks et al. [[1]](https://doi.org/10.1186/s12913-016-1789-4) provided comprehensive documentation, making it more accessible for reproducibility.

Demir et al. [[6]](https://doi.org/10.1057/hs.2016.1) also supports the key findings of our Acute Stroke Unit DES project whereby increasing bed capacity significantly reduces patient delays. Their model, though not stroke-specific, showed that even modest increases in demand led to substantial rises in required bed numbers, echoing our model's non-linear relationship between capacity and delays. It reinforces the broader conclusion that bed availability is a critical driver of performance in acute healthcare systems.

We chose not to use a large language model in the recreation process, as we wanted to build our simulation and coding skills independently. In retrospect, we would include a rehab unit resource and prioritise structured pseudocode earlier to improve logic validation.


### Future Work Directions

| **Area**                  | **Future Work Description**                                                                 |
|---------------------------|----------------------------------------------------------------------------------------------|
| Rehab Unit Modelling      | Introduce a ward-level rehab unit as a downstream resource with its own LOS.               |
| Transfer Logic Expansion  | Allow patients to transfer from external units directly into the rehab unit.                |
| Streamlit App Completion  | Finalise remaining build of Streamlit app functionality   |
| Input Parameters          | Allow the user of a CSV file to alter parameters   |

*Table 7. Future work*

---

### Conclusion

This project successfully replicated the core logic of a published acute stroke DES model, validating key findings around bed capacity and patient delays. Through iterative development and critical reflection, we deepened our understanding of healthcare simulation. The Monks et al. [[1]](https://doi.org/10.1186/s12913-016-1789-4) paper offered a uniquely clear and practical foundation its real-world focus and transparent reporting made it the ideal choice for this DES project.

---

### References

1. Monks T, Worthington D, Allen. M, Pitt M, Stein K and James M (2016). A modelling tool for capacity planning in acute and community stroke services. *BMC Health Services Research*. 2016;16:530. Available from: https://doi.org/10.1186/s12913-016-1789-4 (Accessed: 1st March 2025).

2. Monks T. (2025). *stochastic_systems*. Available from: https://github.com/health-data-science-OR/stochastic_systems (Accessed: 2nd March 2025).

3. Streamlit documentation (no date). Available from: https://docs.streamlit.io/ (Accessed: 20th March 2025).

4. Law AM. (2015) *Simulation Modeling and Analysis*. 5th ed. New York: McGraw-Hill; 2015. Chapter 3.4.4, Statistical Capabilities; p. 190–191. Available from: https://industri.fatek.unpatti.ac.id/wp-content/uploads/2019/03/108-Simulation-Modeling-and-Analysis-Averill-M.-Law-Edisi-5-2014.pdf (Accessed 26th March 2025).

5. Yakutcan U, Demir E, Hurst JR, Taylor PC, and Ridsdale HA. (2021). Operational modeling with health economics to support decision making for COPD patients. *Health Services Research*. 2021;56(6):1271–1280. Available from: https://doi.org/10.1111/1475-6773.13652 (Accessed: 20th March 2025).

6. Demir E, Gunal MM and Southern D. (2017). Demand and capacity modelling for acute services using discrete event simulation. *Health Systems*. 2017;6(1):33–40. Available from: https://doi.org/10.1057/hs.2016.1 (Accessed: 20th March 2025).